## Define Parks

In [1]:
# Set names as numbers for ease-of-use
park_names = [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48]

2×24 Matrix{Int64}:
  1   2   3   4   5   6   7   8   9  …  16  17  18  19  20  21  22  23  24
 25  26  27  28  29  30  31  32  33     40  41  42  43  44  45  46  47  48

## Helper Functions

In [2]:
# HELPER FUNCTION: returns the cycle containing the city START.
function getSubtour(x,start)
    subtour = [start]
    while true
        j = subtour[end]
        for k in park_names
            if x[k,j] == 1
                push!(subtour,k)
                break
            end
        end
        if subtour[end] == start
            break
        end
    end
    return subtour
end

# HELPER FUNCTION: returns a list of all cycles
function getAllSubtours(x)
    nodesRemaining = park_names
    subtours = []
    while length(nodesRemaining) > 0
        subtour = getSubtour(x,nodesRemaining[1])
        push!(subtours, subtour)
        nodesRemaining = setdiff(nodesRemaining,subtour)
    end
    return subtours
end
;

In [3]:
using CSV, DataFrames
c = CSV.File("./arc_data.csv", header=0) |> Tables.matrix
;

## MTZ Optimization Model for TSP

In [4]:
using JuMP, Gurobi, NamedArrays
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

N = size(c, 1)

@variable(m, x[park_names,park_names], Bin)                                      # must formulate as IP this time
@constraint(m, c1[j in park_names], sum( x[i,j] for i in park_names ) == 1)      # one out-edge
@constraint(m, c2[i in park_names], sum( x[i,j] for j in park_names ) == 1)      # one in-edge
@constraint(m, c3[i in park_names], x[i,i] == 0 )                            # no self-loops
@objective(m, Min, sum( x[i,j]*c[i,j] for i in park_names, j in park_names ))   # minimize total cost
                                    
# MTZ variables and constraints
@variable(m, u[park_names])
@constraint(m, c4[i in park_names, j in park_names[2:end]], u[i] - u[j] + N*x[i,j] <= N-1 )

optimize!(m)
xx = value.(x)
subtours = getAllSubtours(xx) 
display(subtours)
println("Tour length: ", objective_value(m), " seconds")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-13


1-element Vector{Any}:
 [1, 26, 14, 2, 42, 33, 12, 6, 16, 23  …  9, 3, 7, 22, 39, 46, 4, 43, 44, 1]

Tour length: 828885.0 seconds


## Suggested Solution

In [5]:
# Redefine park / location names to be easily readable
park_names = ["UW-Madison",
"Acadia National Park",
"Arches National Park",
"Badlands National Park",
"Big Bend National Park",
"Biscayne National Park",
"Black Canyon of the Gunnison National Park",
"Bryce Canyon National Park",
"Canyonlands National Park",
"Capitol Reef National Park",
"Carlsbad Caverns National Park",
"Congaree National Park",
"Crater Lake National Park",
"Cuyahoga Valley National Park",
"Death Valley National Park",
"Everglades National Park",
"Gateway Arch National Park",
"Glacier National Park",
"Grand Canyon National Park",
"Grand Teton National Park",
"Great Basin National Park",
"Great Sand Dunes National Park",
"Great Smoky Mountains National Park",
"Guadalupe Mountains National Park",
"Hot Springs National Park",
"Indiana Dunes National Park",
"Joshua Tree National Park",
"Kings Canyon National Park",
"Lassen Volcanic National Park",
"Mammoth Cave National Park",
"Mesa Verde National Park",
"Mount Rainier National Park",
"New River Gorge National Park",
"North Cascades National Park",
"Olympic National Park",
"Petrified Forest National Park",
"Pinnacles National Park",
"Redwood National Park",
"Rocky Mountain National Park",
"Saguaro National Park",
"Sequoia National Park",
"Shenandoah National Park",
"Theodore Roosevelt National Park",
"Voyageurs National Park",
"White Sands National Park",
"Wind Cave National Park",
"Yellowstone National Park",
"Yosemite National Park",
"Zion National Park"]
;

In [49]:
# Describes optimal solution!
sol_str = ""
for loc in subtours
    for num in loc
        sol_str = sol_str * " → " * string(park_names[num]) * " "
    end
end
println(sol_str)

 → UW-Madison  → Indiana Dunes National Park  → Cuyahoga Valley National Park  → Acadia National Park  → Shenandoah National Park  → New River Gorge National Park  → Congaree National Park  → Biscayne National Park  → Everglades National Park  → Great Smoky Mountains National Park  → Mammoth Cave National Park  → Gateway Arch National Park  → Hot Springs National Park  → Big Bend National Park  → Guadalupe Mountains National Park  → Carlsbad Caverns National Park  → White Sands National Park  → Saguaro National Park  → Joshua Tree National Park  → Death Valley National Park  → Yosemite National Park  → Kings Canyon National Park  → Sequoia National Park  → Pinnacles National Park  → Lassen Volcanic National Park  → Redwood National Park  → Crater Lake National Park  → Olympic National Park  → Mount Rainier National Park  → North Cascades National Park  → Glacier National Park  → Yellowstone National Park  → Grand Teton National Park  → Great Basin National Park  → Capitol Reef National